In [1]:
!pip install owlready2
import xml.etree.ElementTree as ET
from owlready2 import *
from math import ceil

In [2]:
tree = ET.parse('tvshows.xml')
root = tree.getroot()

persons = []
mediaCompanies = []
mediaConglomerates = []
TVChannels = []
advertisingAgencies = []
brands = []
products = []
actors = []
productionHouses = []
characters = []
shows = []
sequelPrequelSpinoffs = []
slots = []

# extracting XML data elements
for child in root:
  if child.tag == 'MediaCompany':
    mediaCompanies.append(child)

  elif child.tag == 'MediaConglomerate':
    mediaConglomerates.append(child)

  elif child.tag == 'TVChannel':
    TVChannels.append(child)

  elif child.tag == 'AdvertisingAgency':
    advertisingAgencies.append(child)

  elif child.tag == 'Brand':    
    brands.append(child)

  elif child.tag == 'Product':
    products.append(child)

  elif child.tag == 'Person':
    persons.append(child)

  elif child.tag == 'Actor':
    actors.append(child)

  elif child.tag == 'ProductionHouse':
    productionHouses.append(child)

  elif child.tag == 'Character':
    characters.append(child)

  elif child.tag == 'Show':
    shows.append(child)

  elif child.tag == 'SequelPrequelSpinoff':
    sequelPrequelSpinoffs.append(child)

  elif child.tag == 'Slot':
    slots.append(child)

In [3]:
o = get_ontology('file://tvshows.owl').load()

# adding media companies
for med in mediaCompanies:
  i = o.MediaCompany(med.attrib['mcID'])
  i.hasID.append(med.attrib['mcID'])
  i.hasName = med.attrib['mcName']

# adding media conglomerates
for medCon in mediaConglomerates:
  i = o.MediaConglomerate(medCon.attrib['mcgID'])
  i.hasID.append(medCon.attrib['mcgID'])
  i.hasName = medCon.attrib['mcgName']
  for listing in medCon:
    j = o[listing.attrib['medCompID']]
    i.owns.append(j)

# adding tv channels
for TVC in TVChannels:
  i = o.TVChannel(TVC.attrib['channelID'])
  i.hasID.append(TVC.attrib['channelID'])
  i.hasName = TVC.attrib['channelName']
  try:
    j = o[TVC.find('MediaCompanyRef').attrib['medCompID']]
    i.ownedBy.append(j)
  except:
    pass

# adding advertising agencies
for adv in advertisingAgencies:
  i = o.AdAgency(adv.attrib['advAgencyID'])
  i.hasID.append(adv.attrib['advAgencyID'])
  i.hasName = adv.attrib['name']

# adding brands
for bra in brands:
  i = o.Brand(bra.attrib['brandID'])
  i.hasID.append(bra.attrib['brandID'])
  i.hasName = bra.attrib['name']

# adding products
for pro in products:
  i = o.Product(pro.attrib['prodID'])
  i.hasID.append(pro.attrib['prodID'])
  i.hasName = pro.attrib['prodName']

  j = o[pro.find('BrandRef').attrib['brandID']]
  i.ownedBy.append(j)

  price = float(pro.find('Price').find('Value').text)
  i.price = price

  proType = o.ProductType()
  i.hasType.append(proType)
  
  cat = pro.find('Type/Category').attrib['categoryName']
  proType.hasCategory.append(cat)

  subCat = pro.find('Type/Subcategory').attrib['subcategoryName']
  proType.hasSubcategory.append(subCat)

# adding persons
for per in persons:
  i = o.Person(per.attrib['personID'])
  i.hasID.append(per.attrib['personID'])

  try:
    i.age = int(per.attrib['age'])
  except:
    pass

  try:
    i.sex = per.attrib['sex']
  except:
    pass
  
  con = o.ContactInfo()
  i.hasContactInfo.append(con)
  try:
    con.hasPhoneNumber = int(per.find('./ContactInfo/Phone').attrib['phoneNumber'])
  except:
    pass
  try:
    con.hasEmail = per.find('./ContactInfo/Email').text
  except:
    pass

# adding actors
for act in actors:
  i = o.Actor(act.attrib['actorID'])
  i.hasID.append(act.attrib['actorID'])

# adding production houses
for proHou in productionHouses:
  i = o.ProductionHouse(proHou.attrib['phID'])
  i.hasID.append(proHou.attrib['phID'])
  i.hasName = proHou.attrib['phName']

  con = o.ContactInfo()
  i.hasContactInfo.append(con)
  try:
    con.hasPhoneNumber = int(proHou.find('./ContactInfo/Phone').attrib['phoneNumber'])
  except:
    pass
  try:
    con.hasEmail = proHou.find('./ContactInfo/Email').text
  except:
    pass

# adding characters
for cha in characters:
  i = o.Character(cha.attrib['charID'])
  i.hasID.append(cha.attrib['charID'])

# adding shows
for sho in shows:
  i = o.TVShow(sho.attrib['showID'])
  i.hasID.append(sho.attrib['showID'])
  i.hasName = sho.attrib['showName']
  
  episodeCount = int(sho.attrib['episodeCount'])
  i.episodeCount.append(episodeCount)

  j = o[sho.attrib['phID']]
  i.bankrolledBy.append(j)

# relating shows
for seq in sequelPrequelSpinoffs:
  i = o[seq.attrib['showID']]
  j = o[seq.attrib['relatedShowID']]
  i.relatedTo.append(j)

# adding tv slots
for slo in slots:
  i = o.Slot(slo.attrib['slotID'])
  i.hasID.append(slo.attrib['slotID'])

  j = o[slo.attrib['channelID']]
  i.deliveredOver.append(j)

  i.slotDate.append(slo.attrib['date'])
  i.slotStartTime.append(slo.attrib['startTime'])
  i.slotEndTime.append(slo.attrib['endTime'])
  i.slotMaxViewership.append(int(slo.attrib['maxViewership']))

  for listing in slo.findall('Telecast'):
    tel = o.ShowTelecast()
    i.telecastedShow.append(tel)
    
    j = o[listing.attrib['showID']]
    tel.showReferred.append(j)

    tel.episodeNumber.append(int(listing.attrib['episodeNumber']))
    tel.episodeDuration.append(int(listing.attrib['duration']))
  
  for listing in slo.findall('Advertisement'):
    adv = o.AdvertisementTelecast(listing.attrib['adID'])
    i.telecastedAd.append(adv)

    adv.hasID.append(listing.attrib['adID'])
    adv.adDuration = int(ceil(float(listing.attrib['adDuration'])))
    adv.adFee = float(listing.attrib['adFee'])

o.save(file = "tvshowsUpdatedWithXMLData.owl", format = "rdfxml")

o2 = get_ontology('file://tvshowsInferences.owl').load()
with o2:
  sync_reasoner(infer_property_values = True)
o2.save(file = "tvshowsInferences.owl", format = "rdfxml")

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/lib/python3.7/dist-packages/owlready2/hermit:/usr/local/lib/python3.7/dist-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmps9xfklwo -Y


* Owlready * Adding relation tvshows.BC321 relatedTo tvshows.YC321
* Owlready * Adding relation tvshows.TINTIN1 relatedTo tvshows.TINTIN2


* Owlready2 * HermiT took 3.8049583435058594 seconds
* Owlready * Reparenting tvshows.Actor: {owl.Thing} => {tvshows.Person}
* Owlready * Reparenting tvshows.Character: {owl.Thing} => {tvshows.Person}
* Owlready * Reparenting tvshows.ProductionHouse: {owl.Thing} => {tvshows.Company}
* Owlready * Reparenting tvshows.MediaCompany: {owl.Thing} => {tvshows.Company}
* Owlready * Reparenting tvshows.MediaConglomerate: {owl.Thing} => {tvshows.Company}
* Owlready * Reparenting tvshows.lastName: {owl.DatatypeProperty, owl.FunctionalProperty, tvshows.nameDetail} => {tvshows.nameDetail, owl.FunctionalProperty}
* Owlready * Reparenting tvshows.firstName: {owl.DatatypeProperty, owl.FunctionalProperty, tvshows.nameDetail} => {tvshows.nameDetail, owl.FunctionalProperty}
* Owlready * Reparenting tvshows.slotDate: {owl.DatatypeProperty, tvshows.slotDetails} => {tvshows.slotDetails}
* Owlready * Reparenting tvshows.slotStartTime: {owl.DatatypeProperty, tvshows.slotDetails} => {tvshows.slotDetails}
* Owl